# OpenVaccine: COVID-19 mRNA Vaccine Degradation Prediction Getting Started Notebook

A starter notebook for those planning to build your submission from scratch. Will update with EDA as and when time permits.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import json
from pandas.io.json import json_normalize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Read train and test json

In [ ]:
train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
train.T

Not so intuitive, right. Lets see if we can flatten it.

One thing to note is that for each parameter(5 nos) that is to be output by the model, two lists each of size seq_scored is provided for the parameter and the error in that parameter.

In [ ]:
train_json = [json.loads(line) for line in open('../input/stanford-covid-vaccine/train.json', 'r')]

In [ ]:
for json in train_json:
    json['step'] = list(range(json['seq_scored']))
    json['sequence'] = list(json['sequence'])
    json['structure'] = list(json['structure'])
    json['predicted_loop_type'] = list(json['predicted_loop_type'])
    json['unpaired_probability'] = list(1-sum(np.load('../input/stanford-covid-vaccine/bpps/'+json['id']+'.npy')))

In [ ]:
train = pd.json_normalize(data = train_json, 
                            record_path ='reactivity',  
                            meta =['id','signal_to_noise',
                                  'SN_filter','seq_length','seq_scored']) 
train.rename(columns={0:'reactivity'}, inplace=True)
train['step'] = pd.json_normalize(data = train_json, 
                            record_path ='step'
                                        )
train['unpaired_probability'] = pd.json_normalize(data = train_json, 
                            record_path ='unpaired_probability'
                                        )
train['sequence'] = pd.json_normalize(data = train_json, 
                            record_path ='sequence'
                                        )
train['structure'] = pd.json_normalize(data = train_json, 
                            record_path ='structure'
                                        )
train['predicted_loop_type'] = pd.json_normalize(data = train_json, 
                            record_path ='predicted_loop_type'
                                        )
train['reactivity_error'] = pd.json_normalize(data = train_json, 
                            record_path ='reactivity_error'
                                        )
train['deg_Mg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_Mg_pH10'
                                        )
train['deg_error_Mg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_Mg_pH10'
                                        )
train['deg_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_pH10',
                                        )
train['deg_error_pH10'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_pH10',
                                        )
train['deg_Mg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_Mg_50C',
                                        )
train['deg_error_Mg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_Mg_50C',
                                        )
train['deg_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_50C',
                                        )
train['deg_error_50C'] = pd.json_normalize(data = train_json, 
                            record_path ='deg_error_50C',
                                        )

train.set_index(['id','step'], inplace=True)
train.T

In [ ]:
train.describe()

Repeat the same with test set

In [ ]:
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)
test.T

In [ ]:
test.describe()

### Make dummy submission File

This will be overwritten if everything works fine.

In [ ]:
submission = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv', index_col= 0)
submission.to_csv('submission.csv')
submission.T

You have to make 107 to 130 predictions per input for the 5 parameters (only 3 of these are used for scoring).

## Develop Your model

Seems you will need something like an LSTM that works on sequence data.

In [ ]:
# Dummy

## Train your model

In [ ]:
# Dummy

## Perform your prediction

In [ ]:
# Dummy

## Filling dummy values

Lets use values mean and std from train data as a dummy submission

In [ ]:
train.describe()

In [ ]:
n = submission.shape[0]
reactivity_bar = 0.374922; reactivity_sd = 0.725652
deg_Mg_pH10_bar = 0.446303; deg_Mg_pH10_sd = 0.704172
deg_pH10_bar = 0.446911; deg_pH10_sd = 1.285747
deg_Mg_50C_bar = 0.407030; deg_Mg_50C_sd = 0.868013
deg_50C_bar = 0.425889; deg_50C_sd = 1.122356
submission['reactivity']=np.random.normal(reactivity_bar, reactivity_sd, size=n)
submission['deg_Mg_pH10']=np.random.normal(deg_Mg_pH10_bar, deg_Mg_pH10_sd, size=n)
submission['deg_pH10']=np.random.normal(deg_pH10_bar, deg_pH10_sd, size=n)
submission['deg_Mg_50C']=np.random.normal(deg_Mg_50C_bar, deg_Mg_50C_sd, size=n)
submission['deg_50C']=np.random.normal(deg_50C_bar, deg_50C_sd, size=n)
submission

## Submit

In [ ]:
submission.to_csv('submission.csv')

# EOF